# Introduction

PyTorch [PR 29488](https://github.com/pytorch/pytorch/pull/29488) implements a SVD for low-rank matrices.
Here we compare the low-rank SVD implementation with the full SVD implementation of pytorch.

## User-interfaces

The low-rank and full SVD have the following user-interfaces:
```
torch.svd_lowrank(A, q=6, niter=2, M=None) -> (U, S, V)

svd(A, some=True, compute_uv=True, out=None) -> (U, S, V)
```

## A test with default values

In [3]:
m, n, r = 17, 5, 3  # matrix shape and rank
A = random_lowrank_matrix(m, n, r)

In [17]:
U1, S1, V1 = torch.svd_lowrank(A)
print(f'S1={S1}')
print(f'U1:\n{U1}')
print(f'V1^T:\n{V1.t()}\n')
U2, S2, V2 = torch.svd(A)
print(f'S2={S2}')
print(f'U2:\n{U2}')
print(f'V2^T:\n{V2.t()}')

S1=tensor([0.2338, 0.1937, 0.0564], dtype=torch.float64)
U1:
tensor([[-0.3878, -0.4516,  0.0321],
        [ 0.7447,  0.0968, -0.2437],
        [-0.5172,  0.2004, -0.4313],
        [-0.0663,  0.3149,  0.8497],
        [-0.1525,  0.8046, -0.1777]], dtype=torch.float64)
V1^T:
tensor([[ 0.4575,  0.8885,  0.0343],
        [ 0.5940, -0.3342,  0.7318],
        [ 0.6617, -0.3144, -0.6807]], dtype=torch.float64)

S2=tensor([0.2338, 0.1937, 0.0564], dtype=torch.float64)
U2:
tensor([[-0.3878, -0.4516, -0.0321],
        [ 0.7447,  0.0968,  0.2437],
        [-0.5172,  0.2004,  0.4313],
        [-0.0663,  0.3149, -0.8497],
        [-0.1525,  0.8046,  0.1777]], dtype=torch.float64)
V2^T:
tensor([[ 0.4575,  0.8885,  0.0343],
        [ 0.5940, -0.3342,  0.7318],
        [-0.6617,  0.3144,  0.6807]], dtype=torch.float64)


So, both svd methods give the same singular values and are able to detect the rank of A correctly.
The UV matrices are also the same (up to a sign multiplier).

# Timings and applicability

In [28]:
%timeit torch.svd_lowrank(A)
%timeit torch.svd(A)
A2 = random_lowrank_matrix(10000, 200, 5)
%timeit torch.svd_lowrank(A2)
%timeit torch.svd(A2)

239 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
22 µs ± 56.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
308 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
43.2 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


So, the full-rank SVD implementation out-performs the low-rank SVD implementation for dense matrices. However, the low-rank SVD implementation will be handy for huge sparse matices where the full-rank SVD fails:

In [40]:
A3 = random_sparse_matrix(100000, 10000, density=2/10000.0)
U3,S3,V3=torch.svd_lowrank(A3)
print(f'S3={S3}')
torch.svd(A3)

S3=tensor([1.1147e+00, 9.0710e-01, 4.6210e-01, 4.1026e-01, 2.5382e-01, 1.8031e-07],
       dtype=torch.float64)


RuntimeError: Could not run 'aten::_svd_helper' with arguments from the 'SparseCPUTensorId' backend. 'aten::_svd_helper' is only available for these backends: [CPUTensorId, VariableTensorId].

# Appendix

The auxiliary code used in this notebook. Run this first!

In [33]:
import torch
import random

def random_matrix(rows, columns, *batch_dims, **kwargs):
    """Return rectangular matrix or batches of rectangular matrices.

    Parameters:
      dtype - the data type
      device - the device kind
      singular - when True, the output will be singular
    """
    dtype = kwargs.get('dtype', torch.double)
    device = kwargs.get('device', 'cpu')
    silent = kwargs.get("silent", False)
    singular = kwargs.get("singular", False)
    if silent and not torch._C.has_lapack:
        return torch.ones(rows, columns, dtype=dtype, device=device)

    A = torch.randn(batch_dims + (rows, columns), dtype=dtype, device=device)
    u, _, v = A.svd(some=False)
    s = torch.zeros(rows, columns, dtype=dtype, device=device)
    k = min(rows, columns)
    for i in range(k):
        s[i, i] = float(i + 1) / (k + 1)
    if singular:
        # make matrix singular
        s[k - 1, k - 1] = 0
        if k > 2:
            # increase the order of singularity so that the pivoting
            # in LU factorization will be non-trivial
            s[0, 0] = 0
    return u.matmul(s.expand(batch_dims + (rows, columns)).matmul(v.transpose(-2, -1)))


def random_lowrank_matrix(rank, rows, columns, *batch_dims, **kwargs):
    """Return rectangular matrix or batches of rectangular matrices with
    given rank.
    """
    B = random_matrix(rows, rank, *batch_dims, **kwargs)
    C = random_matrix(rank, columns, *batch_dims, **kwargs)
    return B.matmul(C)

def random_sparse_matrix(rows, columns, density=0.01, **kwargs):
    """Return rectangular random sparse matrix within given density.

    The density of the result approaches to given density as the size
    of the matrix is increased and a relatively small value of density
    is specified but higher than min(rows, columns)/(rows * columns)
    for non-singular matrices.
    """
    dtype = kwargs.get('dtype', torch.double)
    device = kwargs.get('device', 'cpu')
    singular = kwargs.get("singular", False)

    k = min(rows, columns)
    nonzero_elements = max(min(rows, columns), int(rows * columns * density))

    row_indices = [i % rows for i in range(nonzero_elements)]
    column_indices = [i % columns for i in range(nonzero_elements)]
    random.shuffle(column_indices)
    indices = [row_indices, column_indices]
    values = torch.randn(nonzero_elements, dtype=dtype, device=device)
    # ensure that the diagonal dominates
    values *= torch.tensor([-float(i - j)**4 for i, j in zip(*indices)], dtype=dtype, device=device).exp()
    A = torch.sparse_coo_tensor(indices, values, (rows, columns), device=device)
    return A.coalesce()
